# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f31cc54cfd0>
├── 'a' --> tensor([[ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536]])
└── 'x' --> <FastTreeValue 0x7f31cc54c550>
    └── 'c' --> tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                        [ 0.0920, -0.4071,  0.6370, -0.4344],
                        [ 0.1094,  0.8878, -1.2897, -0.0629]])

In [4]:
t.a

tensor([[ 0.7428,  0.6912,  0.3849],
        [-0.4436,  0.8304,  0.3536]])

In [5]:
%timeit t.a

74.6 ns ± 2.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f31cc54cfd0>
├── 'a' --> tensor([[-0.1977, -1.6350, -0.0590],
│                   [ 0.3748,  0.5500, -0.6671]])
└── 'x' --> <FastTreeValue 0x7f31cc54c550>
    └── 'c' --> tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                        [ 0.0920, -0.4071,  0.6370, -0.4344],
                        [ 0.1094,  0.8878, -1.2897, -0.0629]])

In [7]:
%timeit t.a = new_value

74.7 ns ± 1.41 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7428,  0.6912,  0.3849],
               [-0.4436,  0.8304,  0.3536]]),
    x: Batch(
           c: tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7428,  0.6912,  0.3849],
        [-0.4436,  0.8304,  0.3536]])

In [11]:
%timeit b.a

60.5 ns ± 1.88 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9300, -1.9146, -0.8438],
               [ 1.3509,  0.1656,  0.6391]]),
    x: Batch(
           c: tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629]]),
       ),
)

In [13]:
%timeit b.a = new_value

569 ns ± 5.98 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.6 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

296 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

282 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3127e8dcd0>
├── 'a' --> tensor([[[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]],
│           
│                   [[ 0.7428,  0.6912,  0.3849],
│                    [-0.4436,  0.8304,  0.3536]]])
└── 'x' --> <FastTreeValue 0x7f31218717f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.6 µs ± 765 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3128059280>
├── 'a' --> tensor([[ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536],
│                   [ 0.7428,  0.6912,  0.3849],
│                   [-0.4436,  0.8304,  0.3536]])
└── 'x' --> <FastTreeValue 0x7f31211bdb50>
    └── 'c' --> tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                        [ 0.0920, -0.4071,  0.6370, -0.4344],
                 

In [23]:
%timeit t_cat(trees)

38.5 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.3 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4438, -0.4646, -1.3777, -1.2495],
                       [ 0.0920, -0.4071,  0.6370, -0.4344],
                       [ 0.1094,  0.8878, -1.2897, -0.0629]],
              
                      [[ 0.4438, -0.4646, -1.3777, -1.2495],
                       [ 0.0920, -0.4071,  0.6370, -0.4344],
                       [ 0.1094,  0.8878, -1.2897, -0.0629]],
              
                      [[ 0.4438, -0.4646, -1.3777, -1.2495],
                       [ 0.0920, -0.4071,  0.6370, -0.4344],
                       [ 0.1094,  0.8878, -1.2897, -0.0629]],
              
                      [[ 0.4438, -0.4646, -1.3777, -1.2495],
                       [ 0.0920, -0.4071,  0.6370, -0.4344],
                       [ 0.1094,  0.8878, -1.2897, -0.0629]],
              
                      [[ 0.4438, -0.4646, -1.3777, -1.2495],
                       [ 0.0920, -0.4071,  0.6370, -0.4344],
                       [ 0.1094,  0.8878, -1.2897, -0.0629]],

In [26]:
%timeit Batch.stack(batches)

100 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629],
                      [ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629],
                      [ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629],
                      [ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629],
                      [ 0.4438, -0.4646, -1.3777, -1.2495],
                      [ 0.0920, -0.4071,  0.6370, -0.4344],
                      [ 0.1094,  0.8878, -1.2897, -0.0629],
                      [ 0.4438, -0.4646, -1.3777, -1.2495],
                   

In [28]:
%timeit Batch.cat(batches)

178 µs ± 2.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

595 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
